<a href="https://colab.research.google.com/github/Zirescu/self-join-examination/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An examination of LEFT JOIN on a self-joining dataset

The following data and task is from the PostgreSQL Exercises created by Alisdair Owens.
More specifically: https://pgexercises.com/questions/joins/self2.html

The dataset was copied from [Github](https://github.com/AlisdairO/pgexercises/blob/master/database/clubdata-members.sql), modified slightly and then dumped into ChatGPT so that I could just get a CSV of the original to import into Datacamp. 

When doing the above exercise I was not getting the correct results with my solution. 

Once I saw the solution provided it became a _little_ more clear on what I had done wrong. I've replicated the exercise here so that I could delve into it further and maybe help others with Self joining tables and the LEFT JOIN clause.

# Install our data analysis tools

In [109]:
!pip install --quiet duckdb
!pip install --quiet jupysql 
!pip install --quiet duckdb-engine
!pip install --quiet pandas
import duckdb
import pandas as pd
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:
%sql SELECT version()

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


,version()
0,0.7.1


# Our membership dataset
- Recommendedby points to an existing club member who referred the current member, if they weren't recommended by an existing member the value is null.


In [34]:
members = pd.read_csv('https://raw.githubusercontent.com/Zirescu/self-join-examination/main/clubdata-members.csv?token=GHSAT0AAAAAACBUG56MYRJ6LWP5Z4GW4NACZDNE4PA')
members = members.replace('\\N', pd.NA) # to replace the \N with null
members['joindate'] = members['joindate'].astype('datetime64')  # set the joindate column to a datetime value
members['recommendedby'] = members['recommendedby'].astype('Int64') # set recommended by to Int64 (nullable)
members['zipcode'] = members['zipcode'].astype('object')  # set zipcode to object as we could have hypens
members.head(10)

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,<NA>,2012-07-02 12:02:05
1,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,<NA>,2012-07-02 12:08:23
2,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,<NA>,2012-07-03 09:32:15
3,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05
4,5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,1,2012-07-09 10:44:09
5,6,Tracy,Burton,"3 Tunisia Drive, Boston",45678,(822) 354-9973,<NA>,2012-07-15 08:52:55
6,7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,4,2012-07-25 08:59:12
7,8,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,3,2012-07-25 16:02:35
8,9,Stibbons,Ponder,"5 Dragons Way, Winchester",87630,(833) 160-3900,6,2012-07-25 17:09:05
9,10,Owen,Charles,"52 Cheshire Grove, Winchester, 28563",28563,(855) 542-5251,1,2012-08-03 19:42:37


In [110]:
print(members.dtypes)

memid                     int64
surname                  object
firstname                object
address                  object
zipcode                  object
telephone                object
recommendedby             Int64
joindate         datetime64[ns]
dtype: object


**Some observations from the above:**
- Member 1 (Darren Smith) was not referred by any member of the club.
- Member 2 (Tracy Smith) and 3 (Tim Rownam) were also not referred by any member of the club. 
- Member 4 (Janice Joplette) was referred to the club by member 1 (Darren Smith)
- Member 5 (Gerrald Butters) was referred to the club by member 1 (Darren Smith)



**Our task:**
- Generate a list of members (first `firstname` and last name `surname`) and the names of the members that recommended them to the club. 
- Order by `surname`, `firstname`.

**Let's break this down:**
1. We have a total of 30 members so our expected output needs to be 30 as well. 
2. As we don't have a secondary table that references the names of the individuals that recommended the member we will need to join the members table on itself. 
3. We will need to perform a `LEFT JOIN` so that we keep the original member list and supplement it by returning the names of the individuals that provided the recommendation. 
4. Therefore, the left-hand side of the join **must** be the member id `memid` from the right-most table that points to the `recommendedby` in the left-hand table.

**Let's simplify the data down to focus on just the values we care about.** 

In [38]:
# -- A trimmed down view of our data.
%%sql 
SELECT 
  memid,
  surname,
  firstname,
  recommendedby
FROM members
LIMIT 10;

,memid,surname,firstname,recommendedby
0,1,Smith,Darren,NaN
1,2,Smith,Tracy,NaN
2,3,Rownam,Tim,NaN
3,4,Joplette,Janice,1.0
4,5,Butters,Gerald,1.0
5,6,Tracy,Burton,NaN
6,7,Dare,Nancy,4.0
7,8,Boothe,Tim,3.0
8,9,Stibbons,Ponder,6.0
9,10,Owen,Charles,1.0


**PostgreSQL Definitions for LEFT JOIN**

[LEFT JOIN DEFINITION](https://www.postgresql.org/docs/current/queries-table-expressions.html)
- First, an inner join is performed. 
- Then, for each row in T1 that does not satisfy the join condition with any row in T2, a joined row is added with null values in columns of T2. 
- Thus, the joined table always has at least one row for each row in T1.

[LEFT OUTER JOIN](https://www.postgresql.org/docs/15/sql-select.html) 
- returns all rows in the qualified Cartesian product (i.e., all combined rows that pass its join condition), plus one copy of each row in the left-hand table for which there was no right-hand row that passed the join condition. 
- This left-hand row is extended to the full width of the joined table by inserting null values for the right-hand columns. Note that only the JOIN clause's own condition is considered while deciding which rows have matches. 
- Outer conditions are applied afterwards.

A simplified definition from the Datacamp cheat sheet: 
- A left join keeps all of the original records in the left table and returns missing values for any columns from the right table where the joining field did not find a match.

**Let's write our query such that we can examine it for correctness.**

In [36]:
%%sql 
SELECT 
 l.memid AS memid,  -- left-table memid value
 l.surname AS mem_sname,  -- left-table lastname value
 l.firstname AS mem_fname, -- left-table firstname value
 l.recommendedby AS l_recommendedby, -- left-table recommendedby value
 r.memid AS r_memid, -- our right-table memid value which should be the same as the l_recommendedby value
 r.surname AS rec_fname, -- the first name of the person that recommended the member
 r.recommendedby AS r_reommendedby, -- the member id of any person that was recommended by the recommendy
FROM members AS l
LEFT JOIN members AS r
ON r.memid = l.recommendedby -- where member ID is the recommendedby list ID
ORDER BY mem_sname, mem_fname; -- order by surname, firstname

,memid,mem_sname,mem_fname,l_recommendedby,r_memid,rec_sname,rec_fname,r_reommendedby
0,15,Bader,Florence,9.0,9.0,Stibbons,Ponder,6.0
1,12,Baker,Anne,9.0,9.0,Stibbons,Ponder,6.0
2,16,Baker,Timothy,13.0,13.0,Farrell,Jemima,NaN
3,8,Boothe,Tim,3.0,3.0,Rownam,Tim,NaN
4,5,Butters,Gerald,1.0,1.0,Smith,Darren,NaN
5,22,Coplin,Joan,16.0,16.0,Baker,Timothy,13.0
6,36,Crumpet,Erica,2.0,2.0,Smith,Tracy,NaN
7,7,Dare,Nancy,4.0,4.0,Joplette,Janice,1.0
8,28,Farrell,David,NaN,NaN,None,None,NaN
9,13,Farrell,Jemima,NaN,NaN,None,None,NaN


**Let's examine the above result for correctness:**
1. Our output is 30 rows as expected. 
2. The `l_recommendedby` value is equal to `r_memid` so we know that we've grabbed the correct member info. 
3. We can manually confirm that the names match as well. 

**Now that we've confirmed our results we can simplify the query down to remove the extraneous info.**

In [104]:
# Simplified query to display only the columns we need
%%sql 
SELECT 
 l.surname AS mem_sname,  -- left-table lastname value
 l.firstname AS mem_fname, -- left-table firstname value
 r.surname AS rec_sname, -- the last name of the person that recommended the member
 r.firstname AS rec_fname -- the first name of the person that recommended the member
FROM members AS l
LEFT JOIN members AS r
ON r.memid = l.recommendedby -- where member ID is the recommendeby list
ORDER BY mem_sname, mem_fname; -- order by surname, firstname

,mem_sname,mem_fname,rec_sname,rec_fname
0,Bader,Florence,Stibbons,Ponder
1,Baker,Anne,Stibbons,Ponder
2,Baker,Timothy,Farrell,Jemima
3,Boothe,Tim,Rownam,Tim
4,Butters,Gerald,Smith,Darren
5,Coplin,Joan,Baker,Timothy
6,Crumpet,Erica,Smith,Tracy
7,Dare,Nancy,Joplette,Janice
8,Farrell,David,None,None
9,Farrell,Jemima,None,None


___
If you've made it this far thanks for reading! The following is the rabbit hole I went down when I initially got the solution wrong. The error that I had made was related to order of fields in the ON condition. In the majority of the ON conditions I've seen and used, the order hasn't _seemed_ to matter. 

Below is what happens if the fields of the ON condition are swapped. 

In [102]:
# Below is NOT the desired output 
# When the l.memid is on the left-hand side of the equal operator we will 
# return the names of the members that they recommended and not the names of
# who they were recommended by. 
 
%%sql 
SELECT 
 l.memid AS l_memid,
 l.surname AS mem_sname, 
 l.firstname AS mem_fname, 
 l.recommendedby AS l_recommendedby,
 r.recommendedby AS r_recommendedby,
 r.memid AS r_memid,
 r.surname AS rec_sname,
 r.firstname AS rec_fname
FROM members AS l
LEFT JOIN members AS r
ON l.memid = r.recommendedby -- think of this as give me the records where the left most value appears in the right most value 
ORDER BY mem_sname, mem_fname; 

,l_memid,mem_sname,mem_fname,l_recommendedby,r_recommendedby,r_memid,rec_sname,rec_fname
0,15,Bader,Florence,9.0,15.0,24.0,Sarwin,Ramnaresh
1,12,Baker,Anne,9.0,NaN,NaN,None,None
2,16,Baker,Timothy,13.0,16.0,22.0,Coplin,Joan
3,8,Boothe,Tim,3.0,NaN,NaN,None,None
4,5,Butters,Gerald,1.0,5.0,20.0,Genting,Matthew
5,22,Coplin,Joan,16.0,NaN,NaN,None,None
6,36,Crumpet,Erica,2.0,NaN,NaN,None,None
7,7,Dare,Nancy,4.0,NaN,NaN,None,None
8,28,Farrell,David,NaN,NaN,NaN,None,None
9,13,Farrell,Jemima,NaN,13.0,17.0,Pinker,David


**Analyzing the above query:**
1. We end up with 39 rows instead of 30 rows! We'll dig into this further, but we ended up wtih 22 records which matched and 17 that didn't. 
2. As we can see from the above instead of returning the names of the members that were 'recommeded by' we actually returned back the list of members that the current member recommended. Without the additional fields this is more difficult to validate. 

**Example:**
- Member 15 (Florence Bader) was recommended by Member 9 (Ponder Stibbins), but we show Member 24 (Ramnaresh Sarwin) which was who Florence Bader recommended to the club. 

When I originally saw I was wrong and then looked at the solution it was more evident that the order of the ON condition mattered a lot in this case. I switched them around and voila, the correct solution. 

But now my brain was wondering: _"Why was there 39 rows?"_. To figure this out I went back to the documentation on LEFT joins, which honestly didn't make it that much clearer. I could've stopped there but I _needed_ to know why and went looking for the reasons online. 

I didn't find anything that really helped so I went back to the definition and started to experiment to figure out what I think is going on behind the scenes. 

What follows is that thought experiment...

# An examination of a LEFT JOIN 
We can breakdown the steps of the LEFT JOIN with the following three queries using our prior dataset.

## First-part of Left JOIN
- First, an inner join is performed.


In [94]:
# -- Inner JOIN query and results which generates 22 matching records
# -- This will only produce records where the member has been recommended by another member.
%%sql --save first_part
SELECT 
  m1.memid AS memid,
  m1.surname AS surname,
  m1.firstname AS firstname,
  m1.recommendedby AS recommededby
FROM members m1
INNER JOIN members m2
ON m1.memid = m2.recommendedby
ORDER BY m1.surname, m1.firstname

,memid,surname,firstname,recommededby
0,15,Bader,Florence,9.0
1,16,Baker,Timothy,13.0
2,5,Butters,Gerald,1.0
3,13,Farrell,Jemima,NaN
4,13,Farrell,Jemima,NaN
5,20,Genting,Matthew,5.0
6,11,Jones,David,4.0
7,4,Joplette,Janice,1.0
8,4,Joplette,Janice,1.0
9,30,Purview,Millicent,2.0


<h2 id="heading2"> Second-part of Left JOIN operation</h2>  

- Then, for each row in T1 that does not satisfy the join condition with any row in T2, a joined row is added with null values in columns of T2.Thus, the joined table always has at least one row for each row in T1.  

The above sounded like something I had read before in the Datacamp Cheat sheet which was an "ANTI JOIN".
- The anti join chooses records in the first table where a condition is `NOT` met in the second table. It makes use of a `WHERE` clause to use exclude values from the second table.

Breaking that down further, if we take the results of the `first_part` (above) and do an ANTI JOIN that should in theory give us a list of all the records that did not match the INNER JOIN. 

**Anti Join Query**

In [95]:
%%sql --with first_part --save second_part
SELECT 
  memid, 
  surname, 
  firstname,
  recommendedby
FROM members
WHERE memid NOT IN (
	SELECT memid 
	FROM first_part)

,memid,surname,firstname,recommendedby
0,7,Dare,Nancy,4.0
1,8,Boothe,Tim,3.0
2,10,Owen,Charles,1.0
3,12,Baker,Anne,9.0
4,14,Smith,Jack,1.0
5,17,Pinker,David,13.0
6,21,Mackenzie,Anna,1.0
7,22,Coplin,Joan,16.0
8,24,Sarwin,Ramnaresh,15.0
9,26,Jones,Douglas,11.0


**About the Anti Join Query above:**

To find the 17 records that did not match on the `first_part` we need to perform an ANTI JOIN query (essentially a subquery) where we want to find all the records that did not appear in the INNER JOIN query. 

To do that we query our orginal data set but use a WHERE with a subquery to look-up all the records where the `mem_id` did **not** appear in the results of the INNER JOIN (`first_part`). 

## Third-part - UNION ALL the two result sets together

Now that we've got the two result sets we just need to UNION them together. We need to use `UNION ALL` so that we maintain any duplicates.

In [97]:
# COMBINE the two result sets together and keep duplicates
%%sql --with first_part --with second_part
SELECT *
FROM first_part
UNION ALL -- We need to keep the duplicates!
SELECT *
FROM second_part
ORDER BY surname, firstname;

,memid,surname,firstname,recommededby
0,15,Bader,Florence,9.0
1,12,Baker,Anne,9.0
2,16,Baker,Timothy,13.0
3,8,Boothe,Tim,3.0
4,5,Butters,Gerald,1.0
5,22,Coplin,Joan,16.0
6,36,Crumpet,Erica,2.0
7,7,Dare,Nancy,4.0
8,28,Farrell,David,NaN
9,13,Farrell,Jemima,NaN


There we have it, all 39 records which match up with the results from the LEFT JOIN. 

If we hadn't included the ALL with UNION we get only 30 rows as shown below.

In [99]:
# COMBINE the two result sets together without the ALL
%%sql --with first_part --with second_part
SELECT *
FROM first_part
UNION  -- Without ALL!
SELECT *
FROM second_part
ORDER BY surname, firstname;

,memid,surname,firstname,recommededby
0,15,Bader,Florence,9.0
1,12,Baker,Anne,9.0
2,16,Baker,Timothy,13.0
3,8,Boothe,Tim,3.0
4,5,Butters,Gerald,1.0
5,22,Coplin,Joan,16.0
6,36,Crumpet,Erica,2.0
7,7,Dare,Nancy,4.0
8,28,Farrell,David,NaN
9,13,Farrell,Jemima,NaN


**If you've made it this far I applaud you and I hope this has helped shed some light on LEFT JOIN and the order of the ON condition fields.**

___
# An example for how to find out which members have been recommened by another member. 

In [82]:
# This will only produce records where the member has been recommended by 
# another member
%%sql 
SELECT m1.memid AS m1_memid,
m1.surname AS mem_surname,
m1.firstname AS mem_firstname,
m1.recommendedby,
m2.memid AS m2_memid,
m2.surname AS rec_surname,
m2.firstname AS rec_firstname
FROM members m1, members m2
WHERE m2.memid = m1.recommendedby
ORDER BY m1.surname, m1.firstname

,m1_memid,mem_surname,mem_firstname,recommendedby,m2_memid,rec_surname,rec_firstname
0,15,Bader,Florence,9,9,Stibbons,Ponder
1,12,Baker,Anne,9,9,Stibbons,Ponder
2,16,Baker,Timothy,13,13,Farrell,Jemima
3,8,Boothe,Tim,3,3,Rownam,Tim
4,5,Butters,Gerald,1,1,Smith,Darren
5,22,Coplin,Joan,16,16,Baker,Timothy
6,36,Crumpet,Erica,2,2,Smith,Tracy
7,7,Dare,Nancy,4,4,Joplette,Janice
8,20,Genting,Matthew,5,5,Butters,Gerald
9,35,Hunt,John,30,30,Purview,Millicent


The above and the below are equivalent.

In [100]:
# This will only produce records where the member has been recommended by another member
%%sql 
SELECT 
  m1.memid AS m1_memid,
  m1.surname AS mem_surname,
  m1.firstname AS mem_firstname,
  m1.recommendedby,
  m2.memid AS m2_memid,
  m2.surname AS rec_surname,
  m2.firstname AS rec_firstname
FROM members m1
INNER JOIN members m2
ON m2.memid = m1.recommendedby
ORDER BY m1.surname, m1.firstname

,m1_memid,mem_surname,mem_firstname,recommendedby,m2_memid,rec_surname,rec_firstname
0,15,Bader,Florence,9,9,Stibbons,Ponder
1,12,Baker,Anne,9,9,Stibbons,Ponder
2,16,Baker,Timothy,13,13,Farrell,Jemima
3,8,Boothe,Tim,3,3,Rownam,Tim
4,5,Butters,Gerald,1,1,Smith,Darren
5,22,Coplin,Joan,16,16,Baker,Timothy
6,36,Crumpet,Erica,2,2,Smith,Tracy
7,7,Dare,Nancy,4,4,Joplette,Janice
8,20,Genting,Matthew,5,5,Butters,Gerald
9,35,Hunt,John,30,30,Purview,Millicent
